In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from tabulate import tabulate
import os

def mostrar_tabla(dataframe: pd.DataFrame):
    """
    Muestra un DataFrame de Pandas en formato tabular.

    Args:
    - dataframe (pd.DataFrame): El DataFrame a mostrar.
    """
    print(tabulate(dataframe, headers=dataframe.columns, tablefmt='orgtbl'))

In [ ]:
def clasificar_dependencia(nombre: str) -> str:
    """
    Clasifica la dependencia en una de las categorías predefinidas.

    Args:
    - nombre: Nombre de la dependencia.

    Returns:
    - str: Categoría de la dependencia.
    """
    categorias = {
        'PREPARATORIA': ['PREPARATORIA', 'PREPA.'],
        'FACULTAD': ['FACULTAD', 'FAC.'],
        'HOSPITAL': ['HOSPITAL'],
        'CENTRO': ['CENTRO', 'CTRO.', 'C.', 'INVESTIGAC'],
        'ADMIN': ['SECRETARÍA', 'SECRETARIA', 'SRIA.', 'DIRECCIÓN',
                    'DIRECCION', 'DEPARTAMENTO', 'DEPTO.', 'CONTRALORIA',
                    'AUDITORIA', 'TESORERIA', 'ESCOLAR', 'ABOGACÍA', 'JUNTA',
                    'RECTORIA', 'IMAGEN']
    }

    for categoria, palabras in categorias.items():
        for palabra in palabras:
            if palabra in nombre:
                return categoria
    return 'OTRO'

def transformar_columna_fecha(df: pd.DataFrame,
                                nombre_columna: str) -> pd.DataFrame:
    """
    Transforma la columna de fecha en formato datetime.

    Args:
    - df: DataFrame de Pandas.
    - nombre_columna: Nombre de la columna de fecha.

    Returns:
    - pd.DataFrame: DataFrame con la columna de fecha transformada.
    """
    df[nombre_columna] = pd.to_datetime(df[nombre_columna], format="%Y-%m")
    return df

def agregar_tipos(df_crudo: pd.DataFrame) -> pd.DataFrame:
    """
    Añade las columnas 'Tipo' y 'Fecha' a la tabla.

    Args:
    - df_crudo (pd.DataFrame): La tabla a transformar.

    Returns:
    - pd.DataFrame: La tabla transformada
    """
    df_crudo["Fecha"] = pd.to_datetime(df_crudo["anio"].map(str)+ "-"
                        + raw_df["mes"].map(str), format="%Y-%m")
    df_crudo["Tipo"] = df_crudo["dependencia"].apply(clasificar_dependencia)
    df_transformado = df_crudo.drop(columns=['anio', 'mes'])
    return df_transformado


def analizar_por_dependencia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza un análisis de la información por departamento.

    Args:
    - df: DataFrame de Pandas con la información completa.

    Returns:
    - pd.DataFrame: Tabla con el análisis realizado por departamento.
    """
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")
    df["anio"] = df["Fecha"].dt.year

    df_por_dep = df.groupby(["dependencia", "anio"]).agg({
                    'Sueldo Neto': ['sum', 'count', 'mean', 'min', 'max']
                }).reset_index()
    df_por_dep.columns = ['Tipo', 'anio', 'Suma_Total_sueldos',
                            'Conteo_Empleados', 'Promedio_sueldo',
                            'Salario_Minimo', 'Salario_Maximo']
    
    return df_por_dep


def crear_diagrama_caja(nombre_archivo: str, columna: str,
    funcion_agregacion: callable = pd.DataFrame.sum) -> None:
    """
    Crea un diagrama de caja a partir de valores separados por comas.

    Args:
    - nombre_archivo (str): El nombre del archivo.
    - columna (str): La columna para la cual crear el diagrama de caja.
    - funcion_agregacion (callable): La función de agregación a usar.
    """
    df = pd.read_csv(nombre_archivo)
    df_por_tipo = df.groupby([columna, "Fecha"])[["Sueldo Neto"]] \
                    .aggregate(funcion_agregacion)
    df_por_tipo.boxplot(by=columna, figsize=(27, 18))
    plt.xticks(rotation=90)
    plt.savefig(f"img/boxplot_{columna}.png")
    plt.close()


def graficar_por_dependencia(df: pd.DataFrame, dependencia: str) -> None:
    """
    Crea gráficos de línea y de caja para una dependencia específica.

    Args:
    - df (pd.DataFrame): La tabla con los datos.
    - dependencia (str): La dependencia para la cual crear los gráficos.
    """
    df_dep = df[df["dependencia"] == dependencia]

    df_dep.plot(y=["Sueldo Neto"], title=f"Línea de tiempo - {dependencia}")
    plt.savefig(f"img/lt_{dependencia}.png")
    plt.close()

    df_dep.boxplot(column=["Sueldo Neto"], by='dependencia')
    plt.savefig(f"img/bplt_{dependencia}.png")
    plt.close()


def graficar_por_dependencia_archivo(nombre_archivo: str) -> None:
    """
    Crea gráficos de línea y de caja por dependencia a partir de valores
    separados por comas.

    Args:
    - nombre_archivo (str): El nombre del archivo.
    """
    df_completo = pd.read_csv(nombre_archivo)
    os.makedirs('img', exist_ok=True)
    df_por_dependencia = df_completo.groupby(["dependencia",
                "Fecha"])[["Sueldo Neto"]].agg('count').reset_index()
    df_por_dependencia.set_index("Fecha", inplace=True)

    dependencias = df_por_dependencia["dependencia"].unique()
    for dependencia in dependencias:
       plot_by_dep(df_por_dependencia, dependencia)


    df_aux = df_completo.groupby(["Fecha", "dependencia"])[['Sueldo Neto']] \
                        .mean().unstack()
    df_aux.plot(y='Sueldo Neto', legend=False, figsize=(32, 18))
    plt.xticks(rotation=90)
    plt.xlabel('Fecha')
    plt.ylabel('Sueldo Neto')
    plt.savefig("img/promedio_sueldo_por_dependencia.png")
    plt.close()

def realizar_anova(df: pd.DataFrame, formula_ols: str):
    """
    Realiza un análisis de varianza una tabla dada.

    Args:
    - df (pd.DataFrame): La tabla en la cual realizar el análisis.
    - formula_ols (str): La fórmula para el modelo OLS.
    """
    modelo = ols(formula_ols, data=df).fit()
    tabla_anova = sm.stats.anova_lm(modelo, typ=2)

    if tabla_anova["PR(>F)"][0] < 0.005:
        print("Hay diferencias significativas")
        print(tabla_anova)

        # Prueba de Tukey
        tukey = pairwise_tukeyhsd(df['GastoSalarios'], df['Tipo'])
        print(tukey)
    else:
        print("No hay diferencias significativas")

def realizar_anova(nombre_archivo: str):
    """
    Realiza un análisis de varianza para valores separados por comas.

    Args:
    - file_name (str): El nombre del archivo.
    """
    df_completo = pd.read_csv(nombre_archivo)

    df_por_tipo = df_completo.groupby(["Tipo", "Fecha"])[["Sueldo Neto"]] \
                            .sum().reset_index()
    
    df_aux = df_por_tipo.rename(columns={"Sueldo Neto": "GastoSalarios"})
    
    tipos_interes = ["ADMIN", "CENTRO", "OTRO", "HOSPITAL", "PREPARATORIA"]
    df_aux = df_aux[df_aux["Tipo"].isin(tipos_interes)]

    print(df_aux.head())
    print(df_aux.columns)

    realizar_anova(df_aux, "GastoSalarios ~ Tipo")

def analizar_archivo(nombre_archivo: str) -> None:
    """
    Realiza el análisis de valores separados por comas y muestra los resultados.

    Args:
    - nombre_archivo (str): El nombre del archivo a analizar.
    """
    df_completo = pd.read_csv(nombre_archivo)
    df_completo.columns = df_completo.columns.str.strip()  # Eliminar espacios adicionales
    
    mostrar_tabla(df_completo[["dependencia", "Tipo"]].drop_duplicates().head(150))

    df_por_dependencia = df_completo.groupby(["dependencia",
                            "Fecha"])[["Sueldo Neto"]].sum().reset_index()
    df_por_dependencia.set_index("Fecha", inplace=True)

    for dependencia in df_por_dependencia["dependencia"].unique():
       graficar_por_dependencia(df_por_dependencia, dependencia)

    df_aux = df_completo.groupby(["Fecha","dependencia"])[['Sueldo Neto']] \
                        .mean().unstack()
    df_aux.plot(y='Sueldo Neto', legend=False, figsize=(32,18))
    plt.xticks(rotation=90)
    plt.savefig("img/foo.png")
    plt.close()

    df_por_dependencia.boxplot(by ='dependencia', figsize=(32,18))
    plt.xticks(rotation=90)
    plt.savefig("img/boxplot.png")# , bbox_inches='tight')
    plt.close()

    df_por_tipo = df_completo.groupby(["Tipo",
            "Fecha"])[["Sueldo Neto"]].aggregate(pd.DataFrame.sum)# .count()
    df_por_tipo.boxplot(by = 'Tipo', figsize=(18,9))
    plt.xticks(rotation=90)
    plt.savefig("img/boxplot_tipo.png")
    plt.close()

    df_aux = df_por_tipo.reset_index().rename(columns={"Sueldo Neto": "GastoSalarios"})
    print(df_aux.head())

    realizar_anova(df_aux, "GastoSalarios ~ Tipo")

def agregar_tipos(nombre_archivo: str) -> pd.DataFrame:
    """
    Crear una tabla con tipos a partir de valores separados por comas.

    Args:
    - filename (str): El nombre del archivo.

    Returns:
    - pd.DataFrame: La tabla con tipos.
    """
    df_completo = pd.read_csv(nombre_archivo)
    return agregar_tipos(df_completo)

def mostrar_tipos_dependencias() -> None:
    """
    Muestra los tipos de dependencias.
    """
    df_completo = pd.read_csv("csv/typed_uanl.csv")
    mostrar_tabla(df_completo[['dependencia', 'Tipo']] \
        .drop_duplicates().head(150))

def show_salary_and_count_by_dependency_and_date():
    """
    Muestra el salario y el conteo por dependencia y fecha
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    df_complete.columns = df_complete.columns.str.strip()  # Eliminar espacios adicionales
    
    print("Columnas del DataFrame en show_salary_and_count_by_dependency_and_date:", df_complete.columns)
    
    df_by_type = df_complete.groupby(["dependencia", "Fecha"]).agg({
        'Sueldo Neto': ['sum', 'count', 'mean', 'max']
    })
    
    # Imprimir la estructura del DataFrame
    print("Estructura del DataFrame resultante del groupby y agg:", df_by_type.head())
    
    # Asignar nombres de columnas
    df_by_type.columns = ['Total_sueldos', 'Conteo_Empleado', 'Promedio_sueldo', 'Salario_Maximo']
    df_by_type.reset_index(inplace=True)
    df_by_type.set_index("Fecha", inplace=True)
    
    display_dataframe(df_by_type.head(150))


def show_data_by_type_and_date():
    """
    Muestra los datos por tipo y fecha
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    df_by_type = df_complete.groupby(["Tipo", "Fecha"])[["Sueldo Neto"]].sum()
    display_dataframe(df_by_type.head(150))

def show_salary_and_count_by_type_and_date():
    """
    Muestra el salario y el conteo por tipo y fecha.
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    df_complete.columns = df_complete.columns.str.strip()  # Eliminar espacios adicionales
    
    print("Columnas del DataFrame en show_salary_and_count_by_type_and_date:", df_complete.columns)
    
    df_by_type = df_complete.groupby(["Tipo", "Fecha"]).agg({
        'Sueldo Neto': ['sum', 'count', 'mean', 'max']
    })
    
    # Imprimir la estructura del DataFrame
    print("Estructura del DataFrame resultante del groupby y agg:", df_by_type.head())
    
    # Asignar nombres de columnas
    df_by_type.columns = ['Total_sueldos', 'Conteo_Empleado', 'Promedio_sueldo', 'Salario_Maximo']
    df_by_type.reset_index(inplace=True)
    df_by_type.set_index("Fecha", inplace=True)
    
    display_dataframe(df_by_type.head(150))

if __name__ == "__main__":
    typed_df = create_typed_df("csv/uanl.csv")
    display_dataframe(typed_df.head(50))
    typed_df.to_csv("csv/typed_uanl.csv", index=False)

    analyzed_df = analyze_by_department(typed_df)
    analyzed_df.to_csv("csv/analyzed_uanl.csv", index=False)

    mostrar_tipos_dependencias()

    # show_data_by_type_and_date()

    # show_salary_and_count_by_type_and_date()

    # show_salary_and_count_by_dependency_and_date()

    analizar_archivo("csv/typed_uanl.csv")

    crear_diagrama_caja("csv/typed_uanl.csv", 'Tipo', pd.DataFrame.sum)
    #"Tipo")
    
    # create_plot_por_dependencia("csv/typed_uanl.csv")
    
    # anova_1("csv/typed_uanl.csv")


In [ ]:
def analysis(file_name: str) -> pd.DataFrame:
    df = pd.read_csv(file_name)
    df["hab_x_km2"] = df["poblacion_2020"] / df["area_km2"]
    df["hab_x_mi"] = df["poblacion_2020"] / df["area_mi"]
    print(sum(df["poblacion_2020"]))
    return df

df = analysis("csv/estados_limpio.csv")
display_dataframe(df.head())
display_dataframe(df.describe())
print(df["poblacion_2020"].sum())